In [1]:
from whoosh.fields import Schema, TEXT, ID
from whoosh import index
 
#To create an index in a directory, use index.create_in:
 
import os.path
 
if not os.path.exists("indexdir"):
    os.mkdir("indexdir")
     
schema = Schema(title=TEXT(stored=True), path=ID(stored=True), content=TEXT(stored = True))
 
 
ix = index.create_in("indexdir", schema)
 
writer = ix.writer()
writer.add_document(title=u"My document", content=u"This is my python document! hello big world",
                    path=u"/a")
writer.add_document(title=u"Second try", content=u"This is the second example hello world.",
                    path=u"/b")
writer.add_document(title=u"Third time's the charm", content=u"More examples. Examples are many.",
                    path=u"/c")
 
writer.commit()
 
 
from whoosh.qparser import QueryParser
 
with ix.searcher() as searcher:
     query = QueryParser("content", ix.schema).parse("hello world")
     results = searcher.search(query, terms=True)
     print(results[0])
 
     for r in results:
         print (r, r.score)
         # Was this results object created with terms=True?
         if results.has_matched_terms():
            # What terms matched in the results?
            print(results.matched_terms())
         
     # What terms matched in each hit?
     print ("matched terms")
     for hit in results:
        print(hit.matched_terms())
 
      
 
     print ("more_results")
     first_hit = results[0]
     more_results = first_hit.more_like_this("content")
     print (more_results)     
         
     
found = results.scored_length()
if results.has_exact_length():
    print("Scored", found, "of exactly", len(results), "documents")
else:
    low = results.estimated_min_length()
    high = results.estimated_length()
 
    print("Scored", found, "of between", low, "and", high, "documents")    

<Hit {'content': 'This is the second example hello world.', 'path': '/b', 'title': 'Second try'}>
<Hit {'content': 'This is the second example hello world.', 'path': '/b', 'title': 'Second try'}> 2.124137931034483
{('content', b'hello'), ('content', b'world')}
<Hit {'content': 'This is my python document! hello big world', 'path': '/a', 'title': 'My document'}> 1.7906976744186047
{('content', b'hello'), ('content', b'world')}
matched terms
[('content', b'hello'), ('content', b'world')]
[('content', b'hello'), ('content', b'world')]
more_results
<Top 1 Results for Or([Term('content', 'example', boost=0.6588835188105945), Term('content', 'second', boost=0.6588835188105945), Term('content', 'hello', boost=0.5617184491361429), Term('content', 'world', boost=0.5617184491361429)]) runtime=0.002138999999999669>
Scored 2 of exactly 2 documents
